# Manual del programador

Este documento tiene la finalidad de explicar el funcionamiento de los distintos paquetes de funciones que se crearon para llevar a cabo la solución de la problemática planteada por Fundación Teletón.

El documento se dividirá en 3 apartados: Una breve introducción y explicación de la solución general planteada, explicación de las funciones y un ejemplo del funcionamiento.

### Explicación general de la solución planteada

El archivo **`login.py`** sería el código principal. Éste contiene la interfaz del usuario y manda a llamar a los archivos **`DataBaseConection.py`** y **`CertificateFunctions.py`** para poder utilizar las funciones que estos dos archivos contienen. Existen funciones que deben ser manejadas exclusivamente por un administrador, por ejemplo: la creación de certificados e insertar usuarios nuevos a la base de datos. Por ello, estos códigos fueron brindados de manera separada a la solución y deberán ser ejecutados con una persona con el permiso de hacerlo (administrador).

Al utilizar este código, se asume que, tanto el servidor (en este caso fue creado localmente) como la base de datos fueron debidamente inicializados y se cuenta con al menos un usuario dentro de la misma. Una vez que se tiene al servidor, se pueden ejecutar el siguiente código para crear la base de datos y conectarla (o sólo conectarla) con la aplicación proporcionada:

In [ ]:
# Se cargan las librerías necesarias
import hashlib
import DataBaseConection
from CertificateFunctions import cert_gen, check_associate_cert_with_private_key, VerificarVigencia, Hash_document, VerificarPassword

database = DataBaseConection.DataBase(user = "Usuario", password = "Contraseña", db = "Nombre de la base de datos")

En caso de que no se cuente con algún usuario activo en la base de datos se puede correr las siguientes líneas de código para introducir un nuevo usuario:

In [ ]:
nombre_1 = "Nombre1"
password_1 = "Contrasena1"
nomina_1 = "ID"
email_1 = "Correo electronico1"

cert_gen(emailAddress=email_1,
    commonName="",
    countryName="MX",
    localityName="Municipio donde se expide",
    stateOrProvinceName="Estado donde se expide",
    organizationName="Nombre de la organizacion",
    organizationUnitName="Nombre unitario de la organizacion",
    serialNumber= 0,
    validityStartInSeconds = 0,
    validityEndInSeconds = 1*365*24*60*60, #UN AÑO
    KEY_FILE = 'Nombre del archivo de la clave privada.key'
                #Sugerido: "Ejemplo/private"+nomina_1.replace(" ","_")+".key",
    CERT_FILE='Nombre del archivo del certificado.crt'
                #Sugerido: "Ejemplo/Certificado"+nomina_1.replace(" ","_")+".crt")

print("Firma generada")
print("Guarde su llave privada en un lugar aparte")
print("-------------------\n")
print("Subiendo certificado a la base de datos")

database.insert_users(nomina = nomina_1,
                #Contraseña se subirá encriptada
                password=hashlib.sha256(bytes(password_1, encoding="utf-8")).hexdigest(),
                nombre = nombre_1,
                puesto = "Puesto",
                tags = "Informacion relevante (ejemplo: Becario)",
                certificado = 'Nombre del archivo certificado.crt'
                            #Sugerido: f"Ejemplo/Certificado{nomina_1}.crt",
                estatus = "Activo")

### Explicación de las funciones

#### Documento **`DataBaseConection.py`** 

| Funciones  |  Entradas  |  Salidas |  Descripción  |  
|---|---|---|---|
| __init__  | - Usuario de la base de datos. <br> - Contraseña <br> - Nombre de la base de datos| - Mensaje | Función de inicialización de la clase. Se conecta o crea la base de datos con los parámetros proporcionados |
| insert_users  | - ID de la persona <br> - Constraseña creada por la persona <br> - Nombre completo de la persona <br> - Puesto de la persona <br> - Tags: información adicional de la persona <br> -Archivo certificado de la persona <br> - Estatus: Activo/Inactivo  |- Mensaje | Función para insertar usuarios nuevos a la base de datos en la tabla llamada 'users'. Debe ser manejada por administrador  |   
|  insert_documentos | - Hash del documento a firmar <br> - Tipo de documento <br> - Nombre del archivo <br> - Descripción <br> - IDs de las personas que firmarán (puede ser más de una, separados por un punto-coma ';') <br> - Estatus: Activo/Inactivo  | - Mensaje  | Función para insertar documentos nuevos a la base de datos en la tabla llamada 'dcumentos'. Se manda a llamar por la aplicación cuando se selecciona la opción de 'Solicitar firma' |  
| insert_firma  |  - Documento de firma de documento <br> - Hash del documento que se firmó <br> - ID de la persona que firma |  | Función que sube a la tabla 'firmas' el archivo con la firma digital para el documento identificado con su hash, quién lo firma y la fecha y hora de cuando esto ocurre. Se tiene la intención que esta tabla sirva como *logs* para saber y llevar un control de las firmas. <br> Esta función se llama cuando se escoge la opción de 'Firmar' en la aplicación. |   

A continuación se presentan funciones de comandos típicos de una base de datos MySQL:

| Funciones  |  Entradas  |  Salidas |  Descripción  |  
|---|---|---|---|
| select  |  - Nombre de la tabla <br> - La variable a regresar <br> - Variable independiente <br> - Valor de la variable independiente <br> *Opcional:* <br> - Segunda variable independiente <br> - Valor de la segunda variable independiente | - *list*: Todos los valores de la variable dependiente donde se cumple las condiciones de las variables independientes.  |  Función regresa todos los valores de una variable (columna) donde se cumple las condiciones impuestas por las variables independientes. |   
| update  |  - Nombre de la tabla <br> - La variable a actualizar <br> - Valor de la variable a actualizar <br> - Variable independiente <br> - Valor de la variable independiente |  - Mensaje | Función que actualiza un valor de una variable dentro de la base de datos. Esta actualización está sujeta a las condiciones de la variable independiente y su valor. |  
|  delete | - Nombre de la tabla <br> - La variable independiente <br> - Valor de la variable independiente  | - Mensaje  |  Función que elimina toda una fila donde se cumple la condición impuesta. <br> Esta función debe ser manejada por un administrador. | 
| create  | - Nombre de usuario para la base de datos <br> - Contraseña para acceder a la base de datos <br> - Nombre de la base de datos |   | Función que crea la base de datos considerando los parámetros ingresados. <br> La función debe ser ejecutada por un administrador. |

#### Documento **`CertificateFunctions.py`** 

| Funciones  |  Entradas  |  Salidas |  Descripción  |  
|---|---|---|---|
| cert_gen  |  - Correo electrónico del empleado <br> - Nombre completo del empleado <br> - País donde se expide <br> - Municipio donde se expide <br> - Nombre de la organización que expide el certificado <br> - Nombre unitario de la organización <br> - Nombre de serie <br> - Nombre unitario de la organización <br> - Tiempo de vigencia (*default:* un año) <br> - Ubicación y nombre del archivo donde se creará la clave privada <br> - Ubicación y nombre del archivo donde se creará el certificado |   |  Función que crea los certificados para los empleados o los futuros usuarios de la aplicación de firmas. <br> Deberá ser usada por un administrador. |   
| check_associate_cert_with_private_key  |  - Certificado <br> - Archivo de la clave privada  |  - Mensaje | Esta función verifica si la clave privada ingresada está asociada al certificado ingresado. |  
| Hash_document | - Dirección del archivo  | - Hash del documento ingresado  |  Función que genera el *hash* del documento ingresado.| 
| VerificarVigencia  | - Archivo del certificado | - Mensaje <br> - True/False| Función que verifica si el certificado es aún vigente |
| VerificarPassword  | - ID del empleado <br> - Contraseña del empleado <br> - Objeto de la clase DataBase   | - True/False| Función que verifica si la contrasela ingresada corresponde a la contraseña del usuario. Se aplica para permitirle (o no) el acceso a alguien a la plataforma. |

#### Documento **`signVerify.py`** 

| Funciones  |  Entradas  |  Salidas |  Descripción  |  
|---|---|---|---|
| gen_signature  |  - Clave privada de la persona que firma <br> - Hash del documento <br> - Nombre del documento a firmar <br> - ID de la persona que firma | - Archivo binario con la firma <br>  |  Esta función sirve para generar la firma del documento que se necesite. Se usa la función hash SHA-512. |   
| verify  |  - Certificado <br> - Hash del documento que se quiera verificar <br> - Archivo binario que contiene la firma digital del documento <br> - ``` load = True``` (Saber si lo tiene que leer del disco)  |  - True/False | Esta función verifica si la firma ingresada corresponde a una firma para el documento ingresado. | 

#### Documento **`login.py`** 

| Funciones  |  Entradas  |  Salidas |  Descripción  |  
|---|---|---|---|
| main_to_options <br> potions_to_main <br> options_to_sign <br> options_to_verify <br> options_to_request_signature <br> sign_to_options <br> verify_to_options <br> request_signature_to_options  |  |   |  Funciones que sirven para cerrar ventanas emergentes de la aplicación. Sirve para no cerrarlas manualmente. Se usan dentro del código de la interfaz.|   
| main_window  |  |   | Función que contiene a la ventana principal de la aplicación (Ventana de ingreso a la plataforma). |  
| validateLogin | - Nombre de usuario ingresado <br> - Contraseña ingresada| - Mensaje  |  Función que verifica si el usuario y la contraseña corresponden a algún usuario en la base de datos. <br> 
Esta función se encuentra dentro de la función de main_window() y usa a la función VerificaPassword().| 
| options_window  |  |  | Contiene el código de la ventana de la interfaz relacionada con las opciones de 'Solicitar firma', 'Firmar' y 'Verificar'. |
| sign_window  |   | | Función que contiene lo relacionado a la opción de 'Firmar'. |
| select_file <br> select_privateKey <br> select_publicKey |  | - Mensaje  |  Funciones que permiten seleccionar un archivo del dispositivo local. |   
| sign_file |  | - Mensaje  | Función que toma la clave privada y el documento a firmar. Verifica si la clave privada corresponde al certificado del usuario conectado y verifica la vigencia de este certificado. Si no hubo error, se ejecuta la función ```signVerify.gen_signature()```. |  
| check_permit |  | - Mensaje  |  Función que verifica si la persona que intenta firmmar tiene permiso para hacer; es decir, si su ID se encuentra en los *tags* de ese documento.| 
| verify_window  |  |  | Función que contiene el código relacionado a la opción de 'Verificar'. |
| verify_file  |   | | Función que toma el certificado (se extrae la clave pública), el archivo con la firma digital y el documento a verificar. Verifica si el archivo de la firma digital corresponde a el documento ingresado, se usa la función  ```signVerify.verify()```.|
| request_signature_window |  |  |  Función que contiene el código relacionado a la opción de 'Solicitar firma'.| 
| nominas | - IDs de los empleados que se solicita que firmen <br> - Tipo de documento <br> - Descripción |  | Función que usa la función ```DataBase.insert_documentos()``` para cargar a la base de datos un nuevo documento con sus variables. |


### Ejemplo de aplicación

In [1]:
import signVerify
import hashlib
import DataBaseConection
from CertificateFunctions import cert_gen, check_associate_cert_with_private_key, VerificarVigencia, Hash_document, VerificarPassword

In [2]:
database = DataBaseConection.DataBase(user = "root", password = "", db = "teleton")

Conexión exitosa


#### Creando usuarios

##### Creando certificados

In [3]:
nombre_1 = "Jairo Enrique R"
password_1 = "prueba"
nomina_1 = "A01750442"
email_1 = "A01750442@tec.mx"

nombre_2 = "Ramirez Sanchez R"
nomina_2 = "A01752067"
password_2 = "prueba2"
email_2 = "A01752067@tec.mx"


cert_gen(emailAddress=email_1,
    commonName="tec.mx",
    countryName="MX",
    localityName="Monterrey",
    stateOrProvinceName="Nuevo León",
    organizationName="Tecnológico de Monterrey",
    organizationUnitName="organizationUnitName",
    serialNumber=0,
    validityStartInSeconds = 0,
    validityEndInSeconds = 1*365*24*60*60, #UN AÑO
    KEY_FILE = "Ejemplo/private"+nomina_1.replace(" ","_")+".key",
    CERT_FILE="Ejemplo/Certificado"+nomina_1.replace(" ","_")+".crt")

cert_gen(emailAddress=email_2,
    commonName="tec.mx",
    countryName="MX",
    localityName="Monterrey",
    stateOrProvinceName="Nuevo León",
    organizationName="Tecnológico de Monterrey",
    organizationUnitName="organizationUnitName",
    serialNumber=0,
    validityStartInSeconds = 0,
    validityEndInSeconds = 1*365*24*60*60, #UN AÑO
    KEY_FILE = "Ejemplo/private"+nomina_2.replace(" ","_")+".key",
    CERT_FILE="Ejemplo/Certificado"+nomina_2.replace(" ","_")+".crt")

print("Firmas generadas")
print("Guarde su llave privada en un lugar aparte")
print("-------------------\n")
print("Subiendo certificado a la base de datos")

database.insert_users(nomina = nomina_1,
                password=hashlib.sha256(bytes(password_1, encoding="utf-8")).hexdigest(),
                nombre = nombre_1,
                puesto = "Estudiante",
                tags = "IDM",
                certificado = f"Ejemplo/Certificado{nomina_1}.crt",
                estatus = "Activo")

database.insert_users(nomina = nomina_2,
                password = hashlib.sha256(bytes(password_2, encoding="utf-8")).hexdigest(),
                nombre = nombre_2,
                puesto = "Estudiante",
                tags = "IDM",
                certificado = f"Ejemplo/Certificado{nomina_2}.crt",
                estatus = "Activo")

Firmas generadas
Guarde su llave privada en un lugar aparte
-------------------

Subiendo certificado a la base de datos
Jairo Enrique R ha sido añadido a la base de datos.
Ramirez Sanchez R ha sido añadido a la base de datos.


## Solicitar firma

In [4]:
#Primero se sube el documento que se quiere firmar con todos los datos de indentificación
file_1 = "Ejemplo/Doc_1.pdf"

database.insert_documentos(Hash = Hash_document(file_1).hexdigest(), 
                           Tipo = "Libro", 
                           Nombre = "Doc_1.pdf", 
                           Descripcion = "Justine o los infortunios de la virtud",
                           Tags = "Marqués de Sade",
                           Estatus = "Activo")

file_2 = "Ejemplo/Doc_2.pdf"

database.insert_documentos(Hash = Hash_document(file_2).hexdigest(), 
                           Tipo = "Libro", 
                           Nombre = "Doc_2.pdf", 
                           Descripcion = "Suenio de una noche de verano",
                           Tags = "Shakespeare",
                           Estatus = "Activo")

Doc_1.pdf ha sido añadido a la base de datos.
Doc_2.pdf ha sido añadido a la base de datos.


In [5]:
# Se solicitan las firmas de los empleados. 
# Se carga a la base de datos el hash del documento y la nómina de quien lo firmará, se deja vacío por default 
# espacio para el archivo de firma.
database.insert_firma(Hash = Hash_document(file_1).hexdigest(),
                Nomina = nomina_1)

database.insert_firma(Hash = Hash_document(file_1).hexdigest(),
                Nomina = nomina_2)

database.insert_firma(Hash = Hash_document(file_2).hexdigest(),
                Nomina = nomina_1)

50df68716b0074ebd4e5f2ccb5b378d89a8a174128394a57654d4410d4a268e5 ha sido añadido a la base de datos.
50df68716b0074ebd4e5f2ccb5b378d89a8a174128394a57654d4410d4a268e5 ha sido añadido a la base de datos.
529408553f25564439f509c3554f3ad705848bff071f5342825f20a051c29264 ha sido añadido a la base de datos.


## Generar el archivo firmado

In [6]:
# Obtiene el certificado de la base de datos del usuario tomando su nómina
file_1 = "Ejemplo/Doc_1.pdf"
Certificado_1 = database.select(tabla = "users", 
                what = "Certificado", 
                where = "Nomina", 
                value = nomina_1)[0][0]

# Se carga la llave privada del usuario

private_key_1 = "Ejemplo/privateA01750442.key"

#Se verifica la vigencia para saber si es posible firmar el documento

VerificarVigencia(Certificado_1)

#Se valida si la llave privada coincide con el certificado almacenado, es decir, que quien quiera firmar sea quien dice ser.

Match = check_associate_cert_with_private_key(Certificado_1, private_key_1)
print()

if Match:
    # se generar el archivo de firma digital
    file_name = signVerify.gen_signature(private_key_1, bytes(Hash_document(file_1).hexdigest(), 'utf-8'), file_1, nomina_1)
    print(f"Archivo firmado en {file_name}")

    print("\n Cargando la firma a la base de datos")
    database.cargar_firma(Doc_signed = file_name,
                     Hash = Hash_document(file_1).hexdigest(),
                     Nomina = nomina_1)
else:
    print("La llave privada no coincide con el certificado.\nNo puede firmar este documento.")

print("\n-------------------------------\n")

Certificado_2 = database.select(tabla = "users", 
                what = "Certificado", 
                where = "Nomina", 
                value = nomina_2)[0][0]

private_key_2 = "Ejemplo/privateA01752067.key"

VerificarVigencia(Certificado_2)

Match = check_associate_cert_with_private_key(Certificado_2, private_key_2)

if Match:

    file_name = signVerify.gen_signature(private_key_2, bytes(Hash_document(file_1).hexdigest(), 'utf-8'), file_1, nomina_2)
    print(f"Archivo firmado en {file_name}")

    print("\n Cargando la firma a la base de datos")
    database.cargar_firma(Doc_signed = file_name,
                     Hash = Hash_document(file_1).hexdigest(),
                     Nomina = nomina_2)
else:
    print("La llave privada no coincide con el certificado.\nNo puede firmar este documento.")

Certificado vigente

Archivo firmado en Ejemplo/Doc_1.pdf_A01750442_20220421_105711.sign

 Cargando la firma a la base de datos

-------------------------------

Certificado vigente
Archivo firmado en Ejemplo/Doc_1.pdf_A01752067_20220421_105711.sign

 Cargando la firma a la base de datos


## Verificar firma

In [7]:
# Es cargado el documento a verificar
file_1 = "Ejemplo/Doc_1.pdf"
file_2 = "Ejemplo/Doc_2.pdf"

#Se emula que se suben los certificados

#Se cargan los certificados de las personas que se quieren validar
Certificado_1 = "Ejemplo/CertificadoA01750442.crt"
nomina_1 = "A01750442"
Certificado_2 = "Ejemplo/CertificadoA01752067.crt"

# Se extraé de la base de datos el archivo firmado que coincida con el hash del documento y la nómina
f = database.select(tabla = "firmas", 
                what = "Doc_signed", 
                where = "Hash", 
                value = Hash_document(file_1).hexdigest(),
                where_2 = "Nomina",
                value_2 = nomina_1)[0][0]

if f != None:
    # Si el archvio existe, se valida la firma
    open("Ejemplo/temp.sign", "wb").write(f)

    result = signVerify.verify(Certificado_1, bytes(Hash_document(file_1).hexdigest(), 'utf-8'), "Ejemplo/temp.sign", load = True)
    if result:
        print(f"Verificación exitosa. \nEl archivo fue firmado correctamente por {nomina_1}")
    else:
        print(f"Verificación fallida. \nEl archivo no firmado por {nomina_1}")

else:
    # Si el archivo aún no existe, tiene pendiente la firma
    print(f"El archivo aún no cuenta con la firma de {nomina_1}")


Verificación exitosa. 
El archivo fue firmado correctamente por A01750442
